# Imports and functions

In [ ]:
!pip install --quiet plyfile

In [ ]:
import numpy as np
import pandas as pd
import scipy.interpolate as interp
from pathlib import Path
from plyfile import PlyData, PlyElement
from tqdm.notebook import tqdm

In [ ]:
def normalize_z_axis(xyz, xyzi, val):
    dif = np.hstack( ( np.zeros( ( xyz.shape[0], 2 ) ), np.zeros( ( xyz.shape[0], 1 ) ) + val ) )
    xyzi = ( xyzi[0], xyzi[1], xyzi[2]-val, xyzi[3], xyzi[4] )
    xyz = xyz - dif
    return xyz, xyzi

In [ ]:
def interpolate(data, precision, method="linear", mn=None, mx=None, multiplier=1):
    dataMulti = data*multiplier

    if mn is None or mx is None:
      mn, mx = dataMulti.min(axis=0), dataMulti.max(axis=0)

    mn = precision * (10 * mn / (precision * 10)).astype(int)
    mx = precision * (10 * mx / (precision * 10)).astype(int)

    xm = np.arange(mn[0], mx[0] + precision, precision)
    ym = np.arange(mn[1], mx[1] + precision, precision)

    x, y = np.meshgrid(xm, ym)
    z = interp.griddata(dataMulti[:, 0:2], dataMulti[:, 2], (x, y), method=method)

    xyz = data
    xyzi = [x,y,z,xm,ym]
    return xyzi

In [ ]:
def autoRotate(xyz, precision=0.5):

    # eigenvectors cannot be calculated when there are NaN-s in the image -> replace them with max value
    toRotate = xyz
    nanIndices = np.where(np.isnan(toRotate))
    toRotate[nanIndices] = np.nanmax(toRotate[:,2])

    c = np.nanmean(xyz, axis=0)
    data = xyz - c

    nanIndices = np.where(np.isnan(data))
    data[nanIndices] = np.nanmax(data)

    eig = np.linalg.eig( np.dot( np.transpose( data ), data ) )

    # numpy does not automatically order the eigenvectors.
    # We need to order them from the one corresponding to the max eigen value and descending
    order = np.argsort(eig[0])[::-1]

    PCs = np.vstack(( eig[1][:,order[0]],
                      eig[1][:,order[1]],
                      eig[1][:,order[2]] ))

    c = np.nanmean(toRotate, axis=0)
    rotated_xyz = c + (toRotate - c) @ PCs.T

    rotated_x, rotated_y, rotated_z, rotated_xm, rotated_ym = interpolate(rotated_xyz, precision, "linear", None, None, True)
    rotated_xyzi = rotated_x, rotated_y, rotated_z, rotated_xm, rotated_ym

    rotated_xyz, rotated_xyzi = normalize_z_axis( rotated_xyz, rotated_xyzi, np.nanmin( rotated_xyzi[2] ) )
    return rotated_xyz

# Upload and unzip files

Upload raw or zipped .ply file(s) and then run:

In [ ]:
!unzip -o *.zip

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


In [ ]:
!rm *.zip

rm: cannot remove '*.zip': No such file or directory


# Rectify all *.ply* files:

In [ ]:
path = Path('.')
fns = [p for p in path.iterdir() if p.suffix.lower()=='.ply']
fns

[PosixPath('Scaniverse 2024-02-26 101133bicrop.ply')]

In [ ]:
for fn in tqdm(fns):
    print(f'Processing file: {fn}')

    print(f'--> Reading...')
    plydata = PlyData.read(fn)
    xyz = np.stack((plydata['vertex']['x'], plydata['vertex']['y'], plydata['vertex']['z'])).T

    print(f'--> Rotating...')
    rotated_xyz = autoRotate(xyz, precision=0.5)

    print(f'--> Saving...')
    df = pd.DataFrame(rotated_xyz, columns=['x','y','z'])
    df.to_csv(fn.with_suffix('.csv'), float_format='%.4f', columns=['x','y','z'])

  0%|          | 0/1 [00:00<?, ?it/s]

Processing file: Scaniverse 2024-02-26 101133bicrop.ply
--> Reading...
--> Rotating...
--> Saving...


Zip the .csv files for faster download:

In [ ]:
!zip rectified.zip *.csv

  adding: Scaniverse 2024-02-26 101133bicrop.csv (deflated 73%)
